In [8]:
import random
import faker
from datetime import datetime
import time

from azure.cosmos import CosmosClient
import azure.functions as func

import keys

fake = faker.Faker()

client = CosmosClient(url=keys.COSMOS_ENDPOINT,credential=keys.COSMOS_KEY)
database = client.get_database_client('project-db')
# restaurants_container = database.get_container_client('restaurants')
# users_container = database.get_container_client('users')
reviews_container = database.get_container_client('reviews')



positive_reviews = {
    "Italian": ["Loved the authentic pizza!", "Pasta was divine.", "Best tiramisu I've ever had!", "Incredible risotto.", "The gelato was out of this world."],
    "Chinese": ["The dim sum was exceptional!", "Loved the Peking duck.", "Best hot pot experience.", "Amazing dumplings!", "Crispy sweet and sour pork."],
    "Mexican": ["Fantastic tacos!", "Delicious enchiladas.", "Great guacamole.", "Perfect burritos.", "Authentic quesadillas."],
    "Indian": ["Butter chicken was heavenly.", "Amazing biryani.", "Delicious naan bread.", "Tasty tandoori chicken.", "Incredible masala chai."],
    "Japanese": ["Sushi was top-notch.", "Great ramen, very authentic.", "Loved the tempura.", "Delicious teriyaki chicken.", "Best miso soup I've tasted."]
}

neutral_reviews = {
    "Italian": ["Pizza was okay, not great.", "Pasta was average.", "Tiramisu was alright.", "Risotto was mediocre.", "Gelato was decent."],
    "Chinese": ["Dim sum was just fine.", "Peking duck was okay.", "Hot pot was nothing special.", "Dumplings were average.", "Sweet and sour pork was standard."],
    "Mexican": ["Tacos were alright.", "Enchiladas were passable.", "Guacamole was average.", "Burritos were okay.", "Quesadillas were so-so."],
    "Indian": ["Butter chicken was decent.", "Biryani was unremarkable.", "Naan bread was okay.", "Tandoori chicken was average.", "Masala chai was fair."],
    "Japanese": ["Sushi was ordinary.", "Ramen was passable.", "Tempura was average.", "Teriyaki chicken was okay.", "Miso soup was standard."]
}

negative_reviews = {
    "Italian": ["Pizza was undercooked.", "Pasta was too salty.", "Tiramisu lacked flavor.", "Risotto was overcooked.", "Gelato tasted bland."],
    "Chinese": ["Dim sum was disappointing.", "Peking duck was dry.", "Hot pot was bland.", "Dumplings were greasy.", "Sweet and sour pork was soggy."],
    "Mexican": ["Tacos were too spicy.", "Enchiladas were dry.", "Guacamole was tasteless.", "Burritos were soggy.", "Quesadillas were burnt."],
    "Indian": ["Butter chicken was too creamy.", "Biryani was overcooked.", "Naan bread was stale.", "Tandoori chicken was dry.", "Masala chai was weak."],
    "Japanese": ["Sushi was not fresh.", "Ramen was too salty.", "Tempura was oily.", "Teriyaki chicken was burnt.", "Miso soup was watery."]
}
restaurants = [
    {"id": "restaurant_001", "name": "Gusto's Italian", "location": "123 Main St", "cuisine": "Italian", "operational_hours": "9 AM - 9 PM"},
    {"id": "restaurant_002", "name": "Beijing Bites", "location": "456 Oak Ave", "cuisine": "Chinese", "operational_hours": "9 AM - 9 PM"},
    {"id": "restaurant_003", "name": "Carlos Mexican", "location": "Sreet Wallet JT", "cuisine": "Mexican", "operational_hours": "9 AM - 9 PM"},
     {"id": "restaurant_004", "name": "Zanda BAHABI", "location": "Eve Ball St", "cuisine": "Indian", "operational_hours": "9 AM - 9 PM"},
      {"id": "restaurant_005", "name": "Chu Wawa", "location": "Santas Balmas", "cuisine": "Indian", "operational_hours": "9 AM - 9 PM"}
]

def generate_user():
    return {
        "id": fake.uuid4(),
        "name": fake.name(),
        "email": fake.email(),
        "location": fake.city(),
        "preferences": random.choice(["Vegan", "Seafood", "Spicy", "Desserts", "Vegetarian"]),
        "dob": fake.date_of_birth().strftime("%Y-%m-%d")

    }

def generate_review(user_id, restaurant):
    rating = random.randint(1, 5)
    cuisine = restaurant["cuisine"]

    if rating >= 4:
        review_text = random.choice(positive_reviews[cuisine])
    elif rating == 3:
        review_text = random.choice(neutral_reviews[cuisine])
    else:
        review_text = random.choice(negative_reviews[cuisine])

    return {
        "id": fake.uuid4(),
        "restaurant_id": restaurant["id"],
        "user_id": user_id,
        "rating": rating,
        "review_text": review_text,
        "review_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

users = [generate_user() for _ in range(10)]

try:
    while True:
        user = random.choice(users)
        restaurant = random.choice(restaurants)
        review = generate_review(user["id"], restaurant)
        #insert user
        # try:
        #     users_container.create_item(user)
        # except:
        #     print('user not inserted!')
        # #insert restaurant
        # try:
        #     restaurants_container.create_item(restaurant)
        # except:
        #     print('restaurant item not inserted')
        #insert review
        try:
            reviews_container.create_item(review)
        except:
            print('review item not inserted')

        # print(user)
        # print(restaurant)
        print(review)

        time.sleep(5)

except KeyboardInterrupt:
    print("Real-time data generation stopped.")



{'id': '70a6804c-9e8d-402b-9aa2-52695fc731aa', 'restaurant_id': 'restaurant_003', 'user_id': '725a9fbc-23f2-4cf2-b3ca-4cc9d45c0c01', 'rating': 4, 'review_text': 'Authentic quesadillas.', 'review_date': '2023-12-29 09:56:07'}
{'id': '7b3fdd91-e559-4e81-bd4e-2416a1a5324f', 'restaurant_id': 'restaurant_003', 'user_id': '725a9fbc-23f2-4cf2-b3ca-4cc9d45c0c01', 'rating': 3, 'review_text': 'Quesadillas were so-so.', 'review_date': '2023-12-29 09:56:13'}
{'id': '8d20d155-55d1-45f4-b836-6021a8663ef6', 'restaurant_id': 'restaurant_004', 'user_id': '2c70c40e-7a9b-4a42-ae80-991a0743dad1', 'rating': 2, 'review_text': 'Butter chicken was too creamy.', 'review_date': '2023-12-29 09:56:18'}
{'id': '1721eb4c-eeec-4074-8b18-2b33cd2cc9d0', 'restaurant_id': 'restaurant_003', 'user_id': '5b8a604c-392b-4595-bfda-e8343a392f19', 'rating': 3, 'review_text': 'Quesadillas were so-so.', 'review_date': '2023-12-29 09:56:23'}
{'id': '78e78490-4749-43d1-b9a4-139eb4b909ea', 'restaurant_id': 'restaurant_002', 'user_id